In [1]:
import sys
sys.path.append("../src/")
from utils import CVsample, talib, lgb_forcast
from utils import *
df = pd.read_csv("../data/raw_data/train.csv")
weight = pd.read_csv("../data/raw_data/asset_details.csv")

In [2]:
lgb_params = {'boost': 'gbdt'
         ,'num_leaves': 11
         ,'n_estimators':4000
         ,'max_depth': 11
         ,'subsample_for_bin':200000
         ,'learning_rate':0.05
         ,'min_child_samples':20
         ,'n_jobs':-1
         }
cv = 3
plength=0.8
ptrain=0.8
pgap=0.05
cv_method = 'best'
timestamp_fill = False
method = 'drop'
weight = weight
df = df

In [ ]:
models = {}

In [9]:
wPCC_asset_list = {}
def get_model(Asset_ID):
    dft = df[df['Asset_ID'] == Asset_ID].sort_values(by=['timestamp'], ascending=True)
    # sample
    cvs = CVsample(df=dft, ival='timestamp', cv=cv, plength=plength, ptrain=ptrain, pgap=pgap)
    cvs.main()

    # test
    test = dft[dft['timestamp'] > max(cvs.validation_ends)]
    tb = talib(test)
    test_panel = tb.main()

    # 对每个fold, 有多种方法，平均还是最好
    lgbs, wPCC = [], []
    for i in range(0, cv):
        [train, gap, validation] = cvs.get_df(i)

        # msing data
        ms = MsData(train, timestamp_fill=timestamp_fill, method=method, target='Target')
        train = ms.main()

        # feature
        tb = talib(train)
        train_panel = tb.main()

        tb = talib(validation)
        val_panel = tb.main()

        # lightgbm
        lgb = lgb_forcast(train_panel, val_panel, lgb_params)
        lgb.lgb_train()
        lgbs.append(lgb)
        predict_sigma = math.sqrt(lgb.out_params['cov'][0][0])
        real_sigma = math.sqrt(lgb.out_params['cov'][1][1])
        wPCC_i = weight[weight['Asset_ID'] == Asset_ID]['Weight'].iloc[0]*lgb.out_params['cov'][0][1] / (predict_sigma*real_sigma)
        wPCC.append(wPCC_i)
    feacols = [col for col in train_panel.columns if col not in ['Target', 'timestamp']]
    if cv_method == 'best':
        test['pred'] = lgbs[wPCC.index(max(wPCC))].lgr.predict(test_panel[feacols])
    if cv_method == 'avg':
        test['pred'] = lgbs[0].lgr.predict(test_panel[feacols])
        for i in range(1, cv):
            test['pred'] = test['pred']+lgbs[i].lgr.predict(test_panel[feacols])
        test['pred'] = test['pred'] / cv

    # calculate the final wPCC
    test2 = test[(test['Target'].isnull() == False) & (test['pred'].isnull() == False)]
    cov = np.cov(test2['pred'], test2['Target'])
    wPCC_asset = weight[weight['Asset_ID'] == Asset_ID]['Weight'].values[0]*cov[0][1]/(math.sqrt(cov[0][0]*cov[1][1]))
    wPCC_asset_list[Asset_ID] = wPCC_asset
    return lgbs[wPCC.index(max(wPCC))].lgr

In [ ]:
for i in range(0, 14):
    models[i] = get_model(Asset_ID)

In [ ]:
import gresearch_crypto
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

for i, (df_test, df_pred) in enumerate(iter_test):
    for j , row in df_test.iterrows():        
        if new_models[row['Asset_ID']] is not None:
            try:
                model = models[row['Asset_ID']]
                x_test = get_features(row)
                tb = talib(x_test)
                x_test = tb.main()
                y_pred = model.predict(pd.DataFrame([x_test]))[0]
                df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
            except:
                df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
                traceback.print_exc()
        else: 
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0  
    
    env.predict(df_pred)